In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-13"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
15102,2024-11-13,Brasil Nbb,18:00,Flamengo,Botafogo,1.02,11.60,151.5,1.83,1.83,-21.5,1.98,1.70,04UUETsI,0.980392,0.086207,0.546448,0.546448,0.066599,0.4,0.4,NaN,NaN,98.528,8.784445,0.089157,230.736,120.878043,0.523880,85.126,290.202,101.65,396.20,0.0,0.0,0.0,0.0,1.185609,0.000000,0.107603,-0.81,-0.162,-0.123457,0.878402,0.9,0.021598,4.96,0.992,10.685484,0.295505,0.4,0.104495
15103,2024-11-13,Eua Nba,00:00,Golden State Warriors,Dallas Mavericks,1.68,2.27,233.5,1.81,2.04,-3.5,2.04,1.81,8Ik0txFM,0.595238,0.440529,0.552486,0.490196,0.035767,0.2,0.6,NaN,NaN,327.478,112.344787,0.343061,208.074,100.641603,0.483682,184.082,279.840,180.96,384.00,0.0,0.0,0.0,0.0,0.211237,0.084485,0.084485,2.03,0.406,1.674877,0.643255,0.7,0.056745,0.22,0.044,28.863636,0.450679,0.6,0.149321
15104,2024-11-13,Eua Nba,00:00,Portland Trail Blazers,Minnesota Timberwolves,4.05,1.27,220.5,1.80,2.03,9.5,1.84,2.03,4lyWzGxp,0.246914,0.787402,0.555556,0.492611,0.034315,0.6,0.8,NaN,NaN,405.200,247.928123,0.611866,146.874,20.085066,0.136750,331.420,189.204,179.78,175.50,0.0,0.0,0.0,0.0,0.739006,0.084927,0.069432,-2.05,-0.410,-7.439024,0.398053,0.5,0.101947,-1.76,-0.352,-0.767045,0.541766,0.5,-0.041766
15105,2024-11-13,Eua Nba,21:00,Orlando Magic,Indiana Pacers,2.14,1.76,227.5,1.83,1.83,1.5,1.88,1.78,6XXYlkpK,0.467290,0.568182,0.546448,0.546448,0.035472,0.4,0.6,NaN,NaN,262.550,138.645211,0.528072,224.574,92.620897,0.412429,161.544,254.062,166.44,118.69,0.0,0.0,0.0,0.0,0.137795,0.000000,0.038640,1.91,0.382,2.984293,0.666680,0.9,0.233320,-2.40,-0.480,-1.583333,0.494360,0.3,-0.194360
15106,2024-11-13,Eua Nba,21:30,Oklahoma City Thunder,New Orleans Pelicans,1.07,6.34,222.5,1.80,1.86,-15.5,1.95,1.72,QcWMwPyJ,0.934579,0.157729,0.555556,0.537634,0.092308,0.6,0.4,NaN,NaN,154.772,28.087996,0.181480,241.720,101.352265,0.419296,141.896,179.822,183.58,220.88,0.0,0.0,0.0,0.0,1.005790,0.023184,0.088629,-0.23,-0.046,-1.521739,0.755739,0.6,-0.155739,-3.61,-0.722,-7.396122,0.543569,0.2,-0.343569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15245,2024-11-13,Rússia Superliga,11:00,Chelyabinsk,University-Ugra,1.17,4.66,145.5,1.85,1.85,-10.5,1.95,1.72,OYcvtH7a,0.854701,0.214592,0.540541,0.540541,0.069293,0.4,0.6,NaN,NaN,318.270,383.164149,1.203897,485.220,676.985755,1.395214,114.312,489.964,95.91,1680.00,0.0,1.0,0.0,0.0,0.846588,0.000000,0.088629,2.36,0.472,0.360169,0.621656,0.9,0.278344,-3.11,-0.622,-5.884244,0.000000,0.0,0.000000
15246,2024-11-13,Rússia Superliga,12:00,Dome-Springs Izhevsk,Bars Rostov,1.29,3.37,148.5,1.85,1.85,-7.5,1.97,1.71,tf4LPeNI,0.775194,0.296736,0.540541,0.540541,0.071930,0.6,0.2,NaN,NaN,354.096,402.035689,1.135386,270.236,188.105204,0.696078,135.350,319.186,129.93,577.50,0.0,1.0,0.0,0.0,0.631237,0.000000,0.099917,1.50,0.300,0.966667,0.643123,0.8,0.156877,0.03,0.006,395.000000,0.286947,0.2,-0.086947
15247,2024-11-13,Rússia Superliga,13:00,CSKA Moscow 2,Zenit Petersburg 2,1.34,3.06,162.5,1.85,1.85,-7.5,1.95,1.73,jLaWswwm,0.746269,0.326797,0.540541,0.540541,0.073066,0.4,0.6,NaN,NaN,402.686,467.376829,1.160648,272.160,349.853445,1.285470,149.508,310.432,103.70,117.04,0.0,0.0,0.0,0.0,0.552829,0.000000,0.084545,-0.06,-0.012,-28.333333,0.491136,0.4,-0.091136,1.23,0.246,8.373984,0.000000,0.0,0.000000
15248,2024-11-13,Rússia Superliga,13

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Dallas Mavericks,233.5,1.81,0.8413,Over
1,00:00,Eua Nba,Portland Trail Blazers,Minnesota Timberwolves,220.5,1.80,0.7460,Over
2,21:30,Eua Nba,Brooklyn Nets,Boston Celtics,217.5,1.93,0.9800,Over
3,21:30,Eua Nba,New York Knicks,Chicago Bulls,226.5,1.81,0.8828,Over
4,21:30,Eua Nba,Philadelphia 76ers,Cleveland Cavaliers,216.5,1.84,0.8917,Over
5,22:00,Eua Nba,Houston Rockets,Los Angeles Clippers,212.5,1.81,0.8541,Over
6,22:00,Eua Nba,Milwaukee Bucks,Detroit Pistons,217.5,1.84,0.9854,Over
7,22:00,Eua Nba,San Antonio Spurs,Washington Wizards,221.5,1.81,0.9854,Over
